## Rod Cantilever Example

In [ ]:
import numpy as np

import dismech

geom = dismech.GeomParams(rod_r0=0.001,
                          shell_h=0)

material = dismech.Material(density=1200,
                            youngs_rod=2e6,
                            youngs_shell=0,
                            poisson_rod=0.5,
                            poisson_shell=0)

static_2d_sim = dismech.SimParams(static_sim=False,
                                  two_d_sim=False,
                                  use_mid_edge=False,
                                  use_line_search=False,
                                  show_floor=False,
                                  log_data=True,
                                  log_step=1,
                                  dt=1e-2,
                                  max_iter=25,
                                  total_time=1.0,
                                  plot_step=1,
                                  tol=1e-4,
                                  ftol=1e-4,
                                  dtol=1e-2)

env = dismech.Environment()
env.add_force('gravity', g=np.array([0.0, 0.0, -9.81]))

geo = dismech.Geometry.from_txt(
    '../tests/resources/rod_cantilever/horizontal_rod_n21.txt')

robot = dismech.SoftRobot(geom, material, geo, static_2d_sim, env)

### Time Stepping

Contortion requires a simple moving boundary which is implemented as a before_step callback.

In [ ]:
# Fix first 3 and last 3 nodes
start = np.array(
    np.where(robot.state.q[robot.node_dof_indices].reshape(-1, 3)[:, 0] <= 0.01)[0])
end = np.array(
    np.where(robot.state.q[robot.node_dof_indices].reshape(-1, 3)[:, 0] >= 0.09)[0])

robot = robot.fix_nodes(np.union1d(start, end))

def move_and_twist(robot: dismech.SoftRobot, t: float):
    """ Simple example of a moving boundary condition """
    u0 = 0.1
    w0 = 2

    # Does not converge if moved on first time step
    if t == robot.sim_params.dt:
        return robot

    if t < 0.15:
        # 0 = x, 1 = y, 2 = z
        robot = robot.move_nodes(start, u0 * robot.sim_params.dt, 0)
    else:
        robot = robot.twist_edges([0,1], w0 * robot.sim_params.dt)
    return robot


stepper = dismech.ImplicitEulerTimeStepper(robot)
stepper.before_step = move_and_twist

robots = stepper.simulate()

qs = np.stack([robot.state.q for robot in robots])

In [ ]:
# Plot DOF vector on plotly
t = np.arange(robot.sim_params.total_time, step=robot.sim_params.dt)
options = dismech.AnimationOptions(y_lim=[-0.01, 0.01],
                  title='Contortion Cantilever (N=21)',
                  camera_view=(0,0))

fig = dismech.get_interactive_animation_plotly(robot, t, qs, options)
fig.show()